In [1]:
import json

# read file json data_skill_tensor.json

with open('data/train/total_skill_jds.json', 'r') as f:
    total_skill_jds = json.load(f)

f.close()

# Filter Data

In [2]:
import json

with open('data/removed_skill.json', 'r') as f:
    removed_skill = json.load(f)

f.close()

In [3]:
for idx, skills in enumerate(total_skill_jds):
    total_skill_jds[idx] = [s for s in skills if s not in removed_skill]

In [27]:
# # save to train
# import json

# json_path = 'data/train/total_skill_jds_removed_minout.json'
# with open(json_path, 'w') as f:
#     json.dump(total_skill_jds, f, indent=4)

In [1]:
import json

json_path = 'data/train/total_skill_jds_removed_minout.json'

with open(json_path, 'r') as f:
    total_skill_jds = json.load(f)

f.close()

# Augmentation Data

## Pre Augmentation

In [2]:
# valid data
valid_data_path = 'data/valid/skills_valid.json'

def load_valid_data(filename):

    with open(filename, 'r') as f:
        valid_data = json.load(f)

    f.close()

    # total valid skill
    total_valid_skill = 0
    for skill in valid_data.keys():
        total_valid_skill += len(valid_data[skill])

    print(f'Total valid skill: {total_valid_skill}')

    return valid_data

valid_data = load_valid_data(valid_data_path)

Total valid skill: 36806


In [26]:
def get_frequency_skill(skills_jds):

    frequency_skills = {}
    for skills in skills_jds:
        for skill in skills:
            if skill not in list(frequency_skills.keys()):
                frequency_skills[skill] = 1
            else:
                frequency_skills[skill] += 1

     # Sort the dictionary by frequency in descending order
    sorted_frequency_skills = dict(sorted(frequency_skills.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_frequency_skills

In [60]:
def find_frequency_skill_around(skill, k=-1, size=10, total_skill_jds=total_skill_jds):
    frequency = {}
    for skills in total_skill_jds:
        
        if skill in skills:
            # get sublist with size is from center to horizon
            index = skills.index(skill)
            sublist = list()
            start=int
            end=int
            if len(skills) <= size * 2 + 1:
                sublist = skills.copy()
            else:
                start = index - size
                end = index + size
                if start < 0:
                    start = 0
                if end > len(skills):
                    end=len(skills)

                sublist = skills[start:end].copy()

            for s in sublist:
                if s in frequency:
                    frequency[s] += 1
                else:
                    frequency[s] = 1
            
        else:
            continue

    # sort frequency
    frequency = {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)}

    if k == -1:
        return frequency
    else:
        # return top k skill and remove top 1 (skill a)
        return {i: frequency[i] for i in list(frequency)[1:k+1]}

In [42]:
frequency = get_frequency_skill(total_skill_jds)

In [64]:
import math

def group_skills_relevant(skills, size=20):

    skills_copy = skills.copy()
    # short by frequency of skill
    skills_copy = sorted(skills_copy, key=lambda x: frequency[x], reverse=True)

    num_group = math.ceil(len(skills) / size)
    len_last_group = round(math.modf(len(skills) / size)[0] * size)

    sorted_skill = [] # list of list (group)

    for idx in range(num_group):

        if idx != num_group - 1:
            len_group = size
        else:
            sorted_skill.append(skills_copy)

            return sorted_skill

        start_gr = 0
        stake = skills_copy[start_gr]
        
        skills_copy.pop(start_gr)
        
        group_skill = []
        group_skill.append(stake)

        #region block code 1: find skill in list in around skill
        around_skill = find_frequency_skill_around(stake, len_group)
        for i, skill in enumerate(skills_copy):

            # region block code 2: end find relevant skill
            if len(group_skill) == len_group:
                # sorted_skill.append(group_skill)
                break
            # endregion

            if skill in around_skill:
                group_skill.append(skill)
                skills_copy.pop(i)
        #endregion

        if len(group_skill) < len_group and len(group_skill) > 1:

            choose_skill = group_skill[1:].copy()
            for _ in range(len(group_skill)-1):
                #region block code 2
                if len(group_skill) == len_group:
                    break
                #endregion
                
                pivot_skill = max(choose_skill, key=lambda x: around_skill[x])
                choose_skill = [s for s in choose_skill if s != pivot_skill]
                # region block code 1
                around_pivot = find_frequency_skill_around(pivot_skill, len_group-len(group_skill))
                for i, skill in enumerate(skills_copy):
                    
                    # region block code 2
                    if len(group_skill) == len_group:
                        break
                    # endregion

                    if skill in around_pivot:
                        group_skill.append(skill)
                        skills_copy.pop(i)
                #endregion

        # when loop don't find all element in group
        if len(group_skill) < len_group:
            len_last_group = len_last_group + (len_group - len(group_skill))
            
        
        sorted_skill.append(group_skill)
            
    return sorted_skill

In [65]:
example = [
        "sql server",
        "scalability",
        "selenium",
        "css",
        "nunit",
        "powershell",
        "asp.net extensions for ajax",
        "operations",
        "angular",
        "jquery",
        "c#",
        "ajax",
        "cd",
        "quality assurance",
        "vue.js",
        "react",
        "automated testing",
        "business continuity",
        "web development",
        "mstest",
        "jenkins",
        "git",
        "databases",
        "sql",
        "html",
        "mvc",
        "unit testing",
        "tdd",
        ".net",
        "static code analysis"
    ]

group_skills_relevant(example)

[['css',
  'sql',
  'html',
  'react',
  'c#',
  'jquery',
  'cd',
  'angular',
  '.net',
  'vue.js',
  'sql server',
  'git',
  'ajax'],
 ['powershell',
  'operations',
  'jenkins',
  'tdd',
  'unit testing',
  'databases',
  'scalability',
  'automated testing',
  'selenium',
  'quality assurance',
  'web development',
  'mvc',
  'business continuity',
  'asp.net extensions for ajax',
  'nunit',
  'static code analysis',
  'mstest']]

## Function Augmentation

In [66]:
import random
import numpy as np

def shuffle_data(skills):
    # Make a copy of the input list to avoid modifying the original list
    shuffled_skills = skills.copy()
    
    # Shuffle the copy of the list in place
    random.shuffle(shuffled_skills)
    
    return shuffled_skills

def cutout_data(skills, mask_size=5):

    cutout_skills = skills.copy()
    
    mask_size = min(mask_size, len(cutout_skills))
    
    cutout_index = random.sample(range(len(skills)), mask_size)
    
    for idx in cutout_index:
        cutout_skills[idx] = ''
        

    cutout_skills = [x for x in cutout_skills if x != '']
    
    return cutout_skills


In [67]:
def create_new_data(total_skills, num_samples=2100): # all samples are created = num_samples * 4

    if num_samples > len(total_skills):
        num_samples = len(total_skills) 

    skills_augmentation_collection = []
    # pick random sample in skills jbs
    random_indexs = random.sample(range(len(total_skills)), num_samples)
    for index in random_indexs:

        groups_skills = group_skills_relevant(total_skills[index])

        combine_group_shuffle_1 = []
        combine_group_shuffle_2 = []
        combine_group_cut_1 = []
        combine_group_cut_2 = []

        if len(groups_skills) > 2:
            index_cut = random.sample(range(len(groups_skills)), len(groups_skills)-1)
        else:
            index_cut = range(len(groups_skills))

        for idx, group in enumerate(groups_skills):
            # shuffle data
            augment_skill = shuffle_data(group)

            combine_group_shuffle_1 += augment_skill
            combine_group_shuffle_2 += group

            #cutout
            if idx in index_cut:
                combine_group_cut_1 += augment_skill
                combine_group_cut_2 += group


        skills_augmentation_collection.append(combine_group_shuffle_1)
        skills_augmentation_collection.append(combine_group_shuffle_2)
        skills_augmentation_collection.append(combine_group_cut_1)
        skills_augmentation_collection.append(combine_group_cut_2)

    return skills_augmentation_collection

In [68]:
total_augmentation = create_new_data(total_skill_jds, 1000)
len(total_augmentation)

4000

In [72]:
total_augmentation = create_new_data(total_skill_jds, len(total_skill_jds))
total_skill_jds += total_augmentation

In [73]:
len(total_skill_jds)

133405

In [74]:
# save to train
import json

json_path = 'data/train/total_skill_augmentation_jds.json'
with open(json_path, 'w') as f:
    json.dump(total_skill_jds, f, indent=4)

# End